In [7]:
import numpy as np
import pandas as pd
import FinanceDataReader as fdr
from scipy.optimize import minimize

# 주식들의 티커를 리스트로 만듭니다.
stocks = ['AAPL', 'GOOG', 'MSFT', 'AMZN']

# 데이터 프레임을 생성합니다.
df = pd.DataFrame()

# 각 주식의 주가 데이터를 가져옵니다.
for stock in stocks:
    df[stock] = fdr.DataReader(stock, '2010-01-01')['Close']

# 로그 수익률을 계산합니다.
log_returns = np.log(df / df.shift(1))

# 무위험 이자율을 설정합니다. 필요에 따라 이 값을 조절할 수 있습니다.
rf = 0.01

# 목적 함수: 샤프 비율의 음수를 최소화합니다.
def neg_sharpe_ratio(weights):
    portfolio_return = np.sum(log_returns.mean() * weights) * 252
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(log_returns.cov() * 252, weights)))
    sharpe_ratio = (portfolio_return - rf) / portfolio_volatility
    return -sharpe_ratio

# 제약 조건: 가중치의 합이 1이 되어야 합니다.
cons = ({'type':'eq', 'fun':lambda x: np.sum(x) - 1})

# 가중치의 범위: 0과 1 사이입니다.
bounds = tuple((0, 1) for x in range(len(stocks)))

# 초기 가중치: 모든 주식에 대해 동일하게 배분합니다.
init_weights = [1/len(stocks) for x in range(len(stocks))]

# 최적화 과정을 수행합니다.
opt_weights = minimize(neg_sharpe_ratio, init_weights, method='SLSQP', bounds=bounds, constraints=cons)

print("Optimal weights:", opt_weights.x)
df

Optimal weights: [0.59425838 0.         0.17434853 0.23139309]


,AAPL,GOOG,MSFT,AMZN
Date,,,,
2010-01-04,7.643214,15.610239,30.950001,6.695000
2010-01-05,7.656429,15.541497,30.959999,6.734500
2010-01-06,7.534643,15.149715,30.770000,6.612500
2010-01-07,7.520714,14.797037,30.450001,6.500000
2010-01-08,7.570714,14.994298,30.660000,6.676000
...,...,...,...,...
2023-05-24,171.839996,121.639999,313.850006,116.750000
2023-05-25,172.990005,124.349998,325.920013,115.000000
2023-05-26,175.429993,125.430000,332.890015,120.110001
